# Imports

In [1]:
# !pip install -Iv transformers==4.38.1

In [2]:
# !pip install -Iv datasets==2.1.0

In [3]:
# !pip install -Iv tensorflow==2.15.0

In [4]:
# !pip install -Iv keras==3.0.5

In [5]:
# !pip install tf-keras

In [6]:
# !pip install -Iv nltk==3.2.4

In [7]:
# !pip install -Iv contractions==0.1.73

In [8]:
# !pip install -Iv accelerate==0.27.2

In [9]:
import pandas as pd
import os 
import sklearn
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from datasets import Dataset, load_dataset, load_metric
import datasets
import torch
from transformers import AutoModelForSequenceClassification, DataCollatorWithPadding, DistilBertConfig, DistilBertForSequenceClassification, BertConfig, BertForSequenceClassification, create_optimizer, TrainingArguments, Trainer
from transformers.keras_callbacks import KerasMetricCallback
import numpy as np
from sklearn.metrics import accuracy_score
from transformers import pipeline
import csv
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import contractions

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/utils/generic.py:462: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
2024-03-10 17:37:44.893191: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-10 17:37:45.741773: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/utils/generic.py:319: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/h

# Import Data

In [10]:
train_data_path = 'data/train.csv'
test_data_path = 'data/test.csv'

In [11]:
labelled_data = pd.read_csv(train_data_path)
labelled_data.head(5)

,text,stars
0,WILL NEVER COME BACK! HORRIBLE SERVICE & NASTY...,1
1,Terrible food. Terrible service.\n\nThe absol...,1
2,"So, right away if I go into a buffet setting, ...",2
3,I have gotten good cuts from this place. I eve...,3
4,I felt this place was a bit lackluster conside...,2


In [12]:
labelled_data['stars'] = labelled_data['stars'].replace({1: 0, 2: 1, 3: 2})

In [13]:
labelled_data.head(5)

,text,stars
0,WILL NEVER COME BACK! HORRIBLE SERVICE & NASTY...,0
1,Terrible food. Terrible service.\n\nThe absol...,0
2,"So, right away if I go into a buffet setting, ...",1
3,I have gotten good cuts from this place. I eve...,2
4,I felt this place was a bit lackluster conside...,1


In [14]:
# # 1. Expand Contractions using contractions library
# def expand_contractions(text):
#     return contractions.fix(text)

# # Apply expand_contractions function to 'text' column
# labelled_data['text'] = labelled_data['text'].apply(expand_contractions)

# # 2. Lower Case
# labelled_data['text'] = labelled_data['text'].str.lower()

# # 3. Remove Punctuations
# labelled_data['text'] = labelled_data['text'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

# # 4. Remove Extra Spaces
# labelled_data['text'] = labelled_data['text'].apply(lambda x: re.sub(' +', ' ', x.strip()))

In [15]:
# # Download the WordNet resource
# nltk.download('stopwords')

In [16]:
# # 5. Remove Stopwords
# stop_words = set(stopwords.words('english'))
# labelled_data['text'] = labelled_data['text'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))

In [17]:
# # Download the WordNet resource
# nltk.download('wordnet')

In [18]:
# !unzip /home/ec2-user/nltk_data/corpora/wordnet.zip -d /home/ec2-user/nltk_data/corpora/

In [19]:
# # 6. Stemming and Lemmatization
# lemmatizer = WordNetLemmatizer()
# def lemmatize_words(text):
#     return " ".join([lemmatizer.lemmatize(word) for word in text.split()])
# labelled_data['text'] = labelled_data['text'].apply(lemmatize_words)

In [20]:
# labelled_data.head(10)

In [21]:
X_train, X_valid, y_train, y_valid = train_test_split(
    labelled_data['text'], 
    labelled_data['stars'], 
    test_size= 0.2, 
    random_state= 42, 
    stratify= labelled_data['stars']
)

In [22]:
print(y_train.value_counts())
print(y_valid.value_counts())

stars
2    1600
1    1600
0    1600
Name: count, dtype: int64
stars
0    400
2    400
1    400
Name: count, dtype: int64


In [23]:
train_ind = X_train.index.values.tolist()
valid_ind = X_valid.index.values.tolist()

train_data = {'text': [], 'label': []}
valid_data = {'text': [], 'label': []}

for ti in train_ind:
    train_data['text'].append(X_train[ti])
    train_data['label'].append(y_train[ti])

for vi in valid_ind:
    valid_data['text'].append(X_valid[vi])
    valid_data['label'].append(y_valid[vi])
    
print(len(train_data['text']))
print(len(valid_data['text']))

4800
1200


In [24]:
train_dataset = Dataset.from_dict(train_data)
valid_dataset = Dataset.from_dict(valid_data)
data = datasets.DatasetDict({"train": train_dataset,"valid": valid_dataset})

In [25]:
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 4800
    })
    valid: Dataset({
        features: ['text', 'label'],
        num_rows: 1200
    })
})

# Tokenizer

In [26]:
# tokenizer 
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [27]:
tokenized_data = data.map(preprocess_function, batched=True)

Parameter 'function'=<function preprocess_function at 0x7fe1d98d0670> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/5 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  0%|          | 0/2 [00:00<?, ?ba/s]

In [28]:
tokenized_data

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 4800
    })
    valid: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1200
    })
})

# Data Collator

In [29]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Model Building

In [30]:
id2label = {0: "OneStar", 1: "TwoStar", 2: "ThreeStar"}
label2id = {"OneStar": 0, "TwoStar": 1, "ThreeStar": 2}

In [31]:
# config = DistilBertConfig(num_labels= 3, max_position_embeddings= 1024, id2label=id2label, label2id=label2id) # You can also change the architecture using this config class

# model = DistilBertForSequenceClassification(config=config)

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=3, id2label=id2label, label2id=label2id, max_position_embeddings= 1024, ignore_mismatched_sizes=True
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized because the shapes did not match:
- distilbert.embeddings.position_embeddings.weight: found shape torch.Size([512, 768]) in the checkpoint and torch.Size([1024, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(1024, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [33]:
batch_size = 16
num_epochs = 5
# batches_per_epoch = len(tokenized_data["train"]) // batch_size
# total_train_steps = int(batches_per_epoch * num_epochs)
# optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [34]:
metric_name = "accuracy"
model_name = "distilbert-base-uncased"

args = TrainingArguments(
    f"{model_name}-finetuned",
    evaluation_strategy = "epoch",
    lr_scheduler_type= "linear",
    warmup_steps= 500,
    learning_rate=0.0001,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=15,
    weight_decay=0.001,
    metric_for_best_model=metric_name
)

In [35]:
metric = load_metric('accuracy')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [36]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["valid"],
    data_collator= data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train Model

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.842632,0.595000
2,0.901500,0.846545,0.584167
3,0.901500,0.845185,0.598333
4,0.705000,0.968465,0.610000
5,0.396500,1.372641,0.580000
6,0.396500,1.678784,0.590833
7,0.201900,1.871284,0.595833
8,0.201900,2.359342,0.580833
9,0.110300,2.253268,0.605833


Checkpoint destination directory distilbert-base-uncased-finetuned/checkpoint-500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory distilbert-base-uncased-finetuned/checkpoint-1000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory distilbert-base-uncased-finetuned/checkpoint-1500 already exists and is non-empty. Saving will proceed but saved results may be invalid.


# Save Model

In [ ]:
model.save_pretrained('models')

# Inference

In [45]:
test_data = pd.read_csv(test_data_path)
test_data.head(5)

,ID,text
0,1,\n\nDuring my recent company trip to our Tempe...
1,2,\n\nAfter hearing about Pauly D from Jersey Sh...
2,3,\n\nI had high hopes for this restaurant based...
3,4,We experienced overpriced and underwhelming en...
4,5,\n\nThe windows from this company are definite...


In [46]:
output_path = 'outputs/distil_bert_v04.csv'

In [47]:
with open(output_path, "w", newline='') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(["ID","Label"])
    for index, row in test_data.iterrows():
        text_id = row['ID']
        text = row['text']
        inputs_id = tokenizer.encode(text, add_special_tokens=True, return_tensors="pt")
        with torch.no_grad():
            outputs = model(inputs_id)
        # Get the predicted probabilities
        probs = torch.softmax(outputs.logits, dim=1).squeeze()

        # Convert probabilities to labels
        predicted_label = torch.argmax(probs).item()
        
        if predicted_label == 0: 
            conv_pred = 1
        elif predicted_label == 1:
            conv_pred = 2
        else: 
            conv_pred = 3
        
        csv_writer.writerow([text_id,conv_pred])